In [1]:
import tensorflow as tf
import numpy as np
import re
import zipfile
import os
from collections import Counter



In [2]:
file_path = "1661-0.txt"

with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
    text = f.read()

In [ ]:
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

start = text.find(start_marker)
end = text.find(end_marker)

if start != -1 and end != -1:
    text = text[start + len(start_marker):end]

text = text.lower()


In [3]:
tokens = re.findall(r"\b\w+\b", text)
print("Total tokens:", len(tokens))

Total tokens: 109226


In [5]:
counter = Counter(tokens)

# Keep top 10k words
vocab = ["PAD", "UNK"] + [w for w, _ in counter.most_common(10000)]

word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

unk_id = word_to_idx["UNK"]


In [6]:
final_idx = [word_to_idx.get(tok, unk_id) for tok in tokens]
print("Total indexed tokens:", len(final_idx))


Total indexed tokens: 109226


In [7]:
seq_length = 30
sequences = []
next_words = []

for i in range(len(final_idx) - seq_length):
    sequences.append(final_idx[i:i + seq_length])
    next_words.append(final_idx[i + seq_length])


In [8]:
X = np.array(sequences, dtype=np.int32)
y = np.array(next_words, dtype=np.int32)

print(X.shape, y.shape)  # (samples, 30) (samples,)


(109196, 30) (109196,)


In [10]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(10000).batch(512).prefetch(tf.data.AUTOTUNE)


In [12]:
vocab_size = len(word_to_idx)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(30,)),
    tf.keras.layers.Embedding(vocab_size, 128),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dense(vocab_size, activation="softmax")
])


In [13]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 30, 128)        │     1,140,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8914)           │     2,290,898 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,826,130 (14.60 MB)

 Trainable params: 3,826,130 (14.60 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.fit(dataset, epochs=50)

Epoch 1/50
214/214 ━━━━━━━━━━━━━━━━━━━━ 33s 148ms/step - accuracy: 0.0470 - loss: 7.3113
Epoch 2/50
202/214 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.0651 - loss: 6.3751

KeyboardInterrupt: 

In [ ]:
import numpy as np
import time
from IPython.display import clear_output

# ============================================
# STEP 1: Define helper functions first
# ============================================

def suggest_next_words(model, current_text, top_k=5, temperature=0.7):
    """
    Suggests top K next words based on current text
    """
    tokens = current_text.lower().split()
    seed = [word_to_idx.get(w, unk_id) for w in tokens]
    
    # Pad or trim to seq_length
    if len(seed) < seq_length:
        seed = [word_to_idx["PAD"]] * (seq_length - len(seed)) + seed
    else:
        seed = seed[-seq_length:]
    
    # Get predictions
    seed_array = np.array(seed).reshape(1, -1)
    preds = model.predict(seed_array, verbose=0)[0]
    
    # Apply temperature
    preds = np.log(preds + 1e-9) / temperature
    preds = np.exp(preds)
    preds = preds / np.sum(preds)
    
    # Get top K suggestions
    top_indices = np.argsort(preds)[-top_k:][::-1]
    suggestions = [(idx_to_word[idx], preds[idx]) for idx in top_indices]
    
    return suggestions

def generate_text(model, start_text, num_words=50, temperature=1.0):
    """Generate text continuation"""
    tokens = start_text.lower().split()
    seed = [word_to_idx.get(w, unk_id) for w in tokens]
    
    if len(seed) < seq_length:
        seed = [word_to_idx["PAD"]] * (seq_length - len(seed)) + seed
    else:
        seed = seed[-seq_length:]
    
    generated = tokens.copy()
    
    for _ in range(num_words):
        seed_array = np.array(seed).reshape(1, -1)
        preds = model.predict(seed_array, verbose=0)[0]
        
        preds = np.log(preds + 1e-9) / temperature
        preds = np.exp(preds)
        preds = preds / np.sum(preds)
        
        next_id = np.random.choice(len(preds), p=preds)
        next_word = idx_to_word.get(next_id, "UNK")
        generated.append(next_word)
        seed = seed[1:] + [next_id]
    
    return " ".join(generated)

# ============================================
# STEP 2: Live autocomplete interface
# ============================================

def live_autocomplete():
    """
    Shows live suggestions as you type
    """
    print("=" * 70)
    print("⌨️  LIVE AUTOCOMPLETE - Sherlock Holmes Style")
    print("=" * 70)
    print("Type 'DONE' when finished, 'QUIT' to exit\n")
    
    current_text = ""
    
    while True:
        # Clear and redraw
        clear_output(wait=True)
        print("=" * 70)
        print("⌨️  LIVE AUTOCOMPLETE")
        print("=" * 70)
        
        # Show current text
        print(f"\n📝 Your text: {current_text}_")
        
        # Show live suggestions if there's text
        if current_text.strip() and len(current_text.split()) > 0:
            try:
                suggestions = suggest_next_words(model, current_text, top_k=5, temperature=0.7)
                print("\n💡 Next word suggestions:")
                for i, (word, prob) in enumerate(suggestions, 1):
                    print(f"   {i}. {word} ({prob*100:.0f}%)")
                
                # Show preview of full completion
                preview = generate_text(model, current_text, num_words=15, temperature=0.7)
                completion_part = preview[len(current_text):].strip()
                print(f"\n✨ Full preview: {current_text} {completion_part}...")
            except Exception as e:
                print(f"\n⚠️  Error: {e}")
        
        # Get input
        print("\n" + "-" * 70)
        user_input = input("Type word (or DONE/QUIT): ").strip()
        
        if user_input.upper() == 'QUIT':
            print("👋 Goodbye!")
            break
        
        if user_input.upper() == 'DONE':
            # Generate full completion
            print("\n🎯 Generating full text...\n")
            final = generate_text(model, current_text, num_words=50, temperature=0.7)
            print(f"📄 Final text:\n\n{final}\n")
            
            print("\nStart over? (y/n): ", end="")
            if input().lower() != 'y':
                break
            current_text = ""
            continue
        
        # Add word to current text
        if user_input:
            current_text += " " + user_input if current_text else user_input

# ============================================
# STEP 3: Run it!
# ============================================

live_autocomplete()